### Notebook *NB05c – Modelo LSTM con sentimiento FULL (horizonte 1 día)*  
**Autor:** Jesús Daniel Romeral Cortina

**Objetivo:**  
Entrenar y evaluar un modelo LSTM utilizando variables financieras del S&P 500 junto con variables de sentimiento extraídas de noticias financieras, con el fin de analizar si la incorporación de información de sentimiento mejora la capacidad predictiva en la predicción direccional a 1 día frente al baseline sin sentimiento.


In [1]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, balanced_accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix)
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import regularizers


2026-02-04 21:58:38.395753: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
NOMBRE = "LSTM"
TIPO_MODELO = "LSTM"
HORIZONTE = "1d"
USA_SENTIMIENTO = 1

In [ ]:
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [4]:
MODEL_PATH   = "../../datos/sp500_sent_model.csv"
OUT_PATH = "../../resultados/resultados_lstm_1d_SENT_FULL.csv"

In [5]:
df = pd.read_csv(MODEL_PATH, parse_dates=["Date"])
df = df.sort_values("Date").set_index("Date")
df.head()

,Close,High,Low,Open,Volume,Return,Target_1d,Return_5d_forward,Target_5d,ret_lag_1,...,ret_lag_5,ret_ma_5,ret_std_5,ret_ma_10,ret_std_10,sentiment_mean,n_news,has_news,sentiment_filled,n_news_filled
Date,,,,,,,,,,,,,,,,,,,,,
2013-01-02,1462.420044,1462.430054,1426.189941,1426.189941,4202600000,0.025403,0,-0.000957,0,0.016942,...,-0.002440,0.005058,0.015419,0.002286,0.012203,0.470913,4.0,1,0.470913,4.0
2013-01-03,1459.369995,1465.469971,1455.530029,1462.420044,3829730000,-0.002086,1,0.008737,1,0.025403,...,-0.004787,0.005598,0.015030,0.000928,0.011814,0.000000,2.0,1,0.000000,2.0
2013-01-04,1466.469971,1467.939941,1458.989990,1459.369995,3424290000,0.004865,0,0.003805,1,-0.002086,...,-0.001218,0.006815,0.014580,0.002174,0.011468,0.000000,1.0,1,0.000000,1.0
2013-01-07,1461.890015,1466.469971,1456.619995,1466.469971,3304970000,-0.003123,0,0.006013,1,0.004865,...,-0.011050,0.008400,0.012423,0.001313,0.011515,0.000000,2.0,1,0.000000,2.0
2013-01-08,1457.150024,1461.890015,1451.640015,1461.890015,3601600000,-0.003242,1,0.010424,1,-0.003123,...,0.016942,0.004363,0.012231,0.001926,0.011035,0.244576,3.0,1,0.244576,3.0


In [6]:
Y = df["Target_1d"]
X = df.drop(columns=[
    "Target_1d", 
    "Target_5d", 
    "Return_5d_forward",
    "Close",
    "High",
    "Low",
    "Open",
    "Volume",
    "sentiment_mean",  #nan
    "n_news"
])

In [7]:
train_mask = df.index < "2022-01-01"
X_train_raw, X_test_raw = X.loc[train_mask], X.loc[~train_mask]
y_train, y_test = Y.loc[train_mask], Y.loc[~train_mask]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_raw)
X_test_scaled = scaler.transform(X_test_raw)  

y_train_arr = y_train.values
y_test_arr = y_test.values

In [8]:

def make_sequences(X: np.ndarray, y: np.ndarray, lookback: int):
    
    X_seq, y_seq = [], []
    for i in range(lookback, len(X)):
        X_seq.append(X[i - lookback:i])
        y_seq.append(y[i])
    return np.array(X_seq, dtype=np.float32), np.array(y_seq, dtype=np.int32)

lookback = 10  
X_train_seq, y_train_seq = make_sequences(X_train_scaled, y_train_arr, lookback)
X_test_seq,  y_test_seq  = make_sequences(X_test_scaled,  y_test_arr,  lookback)

print(f"Entrenamiento: {X_train_seq.shape}") 
print(f"Prueba (Test): {X_test_seq.shape}")



print("X_train_seq:", X_train_seq.shape, "y_train_seq:", y_train_seq.shape)
print("X_test_seq :", X_test_seq.shape,  "y_test_seq :", y_test_seq.shape)



test_dates_seq = X_test_raw.index[lookback:]
print("Fechas test (seq):", test_dates_seq.min(), "->", test_dates_seq.max())

Entrenamiento: (2257, 10, 13)
Prueba (Test): (534, 10, 13)
X_train_seq: (2257, 10, 13) y_train_seq: (2257,)
X_test_seq : (534, 10, 13) y_test_seq : (534,)
Fechas test (seq): 2022-01-18 00:00:00 -> 2024-03-04 00:00:00


In [9]:
val_size = int(len(X_train_seq) * 0.2)

X_val_seq = X_train_seq[-val_size:]
y_val_seq = y_train_seq[-val_size:]

X_train_seq2 = X_train_seq[:-val_size]
y_train_seq2 = y_train_seq[:-val_size]


In [10]:


n_features = X_train_seq.shape[-1]

model = models.Sequential([
        layers.Input(shape=(lookback, n_features)),
        layers.LSTM(32, return_sequences=False,kernel_regularizer=regularizers.l2(1e-4)),
        layers.Dropout(0.2),
        layers.Dense(16, activation="relu", kernel_regularizer=regularizers.l2(1e-4)),
        layers.Dropout(0.2),
        layers.Dense(1, activation="sigmoid")
    ])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)




model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 32)             │         5,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,433 (25.13 KB)

 Trainable params: 6,433 (25.13 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:

callbacks = [
    EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-5)
]




history = model.fit(
    X_train_seq2, y_train_seq2,
    validation_data=(X_val_seq, y_val_seq),
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 4s 18ms/step - accuracy: 0.5260 - loss: 0.6967 - val_accuracy: 0.5809 - val_loss: 0.6886 - learning_rate: 0.0010
Epoch 2/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5515 - loss: 0.6928 - val_accuracy: 0.5809 - val_loss: 0.6862 - learning_rate: 0.0010
Epoch 3/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5559 - loss: 0.6915 - val_accuracy: 0.5765 - val_loss: 0.6857 - learning_rate: 0.0010
Epoch 4/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5537 - loss: 0.6900 - val_accuracy: 0.5743 - val_loss: 0.6849 - learning_rate: 0.0010
Epoch 5/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5604 - loss: 0.6905 - val_accuracy: 0.5765 - val_loss: 0.6845 - learning_rate: 0.0010
Epoch 6/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.5670 - loss: 0.6858 - val_accuracy: 0.5765 - val_loss: 0.6842 - learning_rate: 0.0010
Epoch 7/100
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5670 - loss: 0.6820 - val_ac

In [12]:
y_proba = model.predict(X_test_seq, verbose=0).ravel()
y_pred  = (y_proba >= 0.58).astype(int)  

metrics = {
    "Modelo": NOMBRE,
    "tipo:modelo": TIPO_MODELO,
    "horizonte": HORIZONTE,
    "usa_sentimiento": USA_SENTIMIENTO,
    "Acc": accuracy_score(y_test_seq, y_pred),
    "B_Acc": balanced_accuracy_score(y_test_seq, y_pred),
    "F1": f1_score(y_test_seq, y_pred),
    "ROC": roc_auc_score(y_test_seq, y_proba),
    "Conf_Matrix": confusion_matrix(y_test_seq, y_pred)
}



In [13]:

df_res = pd.DataFrame([metrics])

df_res.drop(columns="Conf_Matrix").to_csv(OUT_PATH, index=False)

print("Resultados guardados en:", OUT_PATH)
df_res.drop(columns="Conf_Matrix")

Resultados guardados en: ../../resultados/resultados_lstm_1d_SENT_FULL.csv


,Modelo,tipo:modelo,horizonte,usa_sentimiento,Acc,B_Acc,F1,ROC
0,LSTM,LSTM,1d,1,0.509363,0.510451,0.619186,0.537243
